In [7]:
!pip install --upgrade numpy gym
# !pip install numpy==1.23.5
# !pip install gym==0.26.2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [8]:
import gym
import numpy as np
from collections import defaultdict

env = gym.make("Blackjack-v1", natural=False, sab=False)

# Initialize state-action value function and visit counts
Q = defaultdict(lambda: np.zeros(2))
returns = defaultdict(list)
policy = defaultdict(lambda: 0)  # 0 = stick, 1 = hit

# Monte Carlo Control
num_episodes = 100000
gamma = 1.0  # Discount factor

for episode in range(num_episodes):
    state = env.reset()[0]
    episode_data = []
    
    # Generate an episode
    while True:
        action = policy[state] if state in policy else env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
        episode_data.append((state, action, reward))
        state = next_state
        if done:
            break
    
    # Compute returns and update Q values
    G = 0
    visited = set()
    
    for state, action, reward in reversed(episode_data):
        G = gamma * G + reward
        if (state, action) not in visited:
            visited.add((state, action))
            returns[(state, action)].append(G)
            Q[state][action] = np.mean(returns[(state, action)])
    
    # Policy improvement
    for state in Q:
        policy[state] = np.argmax(Q[state])

# Testing the learned policy over multiple games
num_test_episodes = 1000
total_reward = 0

for episode in range(num_test_episodes):
    state = env.reset()[0]
    print(f"\nTest Game {episode + 1} Started! Initial State: {state}")
    
    while True:
        action = policy[state] if state in policy else env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
        
        print(f"State: {state}, Action: {'Hit' if action == 1 else 'Stick'}, Reward: {reward}")
        
        state = next_state
        if done:
            total_reward += reward
            print(f"Game over! Final Reward: {reward}")
            break

# Calculate and display average reward over test episodes
average_reward = total_reward / num_test_episodes
print(f"\nAverage reward over {num_test_episodes} test games: {average_reward:.2f}")


AttributeError: module 'numpy' has no attribute 'bool8'